In [8]:
import time
import os
import sys

from isek.agent.single_agent import SingleAgent
from isek.llm.openai_model import OpenAIModel
from isek.agent.persona import Persona
from dotenv import load_dotenv

def image_process() 

def combination_calculator(n: str, k: str) -> str:
    # non-recursive implementation
    n = int(n)
    k = int(k)
    if k == 0:
        return "1"
    if k > n:
        return "0"
    result = 1
    for i in range(k):
        result = result * (n - i) // (i + 1)
    return str(result)

def add(a: str, b: str) -> str:
    return str(int(a) + int(b))

def subtract(a: str, b: str) -> str:
    return str(int(a) - int(b))

def multiply(a: str, b: str) -> str:
    return str(int(a) * int(b))

def divide(a: str, b: str) -> str:
    return str(int(a) / int(b))

In [20]:
def setup_agent() -> SingleAgent:
    """
    Set up and return a configured ISEK agent.
    """
    # Load environment variables
    load_dotenv()

    # Initialize the OpenAI model
    model = OpenAIModel(
        model_name=os.environ.get("OPENAI_MODEL_NAME"),
        base_url=os.environ.get("OPENAI_BASE_URL"),
        api_key=os.environ.get("OPENAI_API_KEY")
    )

    # Define the persona for the agent (customize as needed)
    eddie_info = {
        "name": "Eddie",
        "bio": "You are a helpful assistant.",
        "lore": "I will ask you a question.",
        "knowledge": "probability code, demystified and so on.",
        "routine": "Report your thoughts, and finish your answer with the following template: FINAL ANSWER: [YOUR FINAL ANSWER]. YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings. If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise. If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise. If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string."
    }
    eddie = Persona.from_json(eddie_info)

    # Create the agent
    eddie_agent = SingleAgent(model=model, persona=eddie)

    # Register the tools with the agent
    eddie_agent.tool_manager.register_tools([combination_calculator, add, subtract, multiply, divide])
    
    return eddie_agent

In [28]:
from datasets import load_dataset

dataset = load_dataset("gaia-benchmark/GAIA",'2023_all')
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['task_id', 'Question', 'Level', 'Final answer', 'file_name', 'file_path', 'Annotator Metadata'],
        num_rows: 301
    })
    validation: Dataset({
        features: ['task_id', 'Question', 'Level', 'Final answer', 'file_name', 'file_path', 'Annotator Metadata'],
        num_rows: 165
    })
})


In [22]:
import os
import sys
import time
import json
from datasets import load_dataset
from tqdm import tqdm
def evaluate_agent(agent: SingleAgent, dataset_split, output_file: str) -> None:
    """
    Evaluate the agent on the given dataset split.
    Saves the evaluation results to a JSON file.

    Parameters:
    - agent: The ISEK agent instance.
    - dataset_split: A Hugging Face dataset split (for example, test split).
    - output_file: Path to save the JSON results.
    """
    results = []
    correct = 0
    total = len(dataset_split)

    # Iterate over each example in the dataset split with a progress bar.
    for example in tqdm(dataset_split, desc="Evaluating"):
        question = example.get("Question")
        ground_truth = example.get("Final answer")

        if not question:
            # Skip examples without a question
            continue

        # Run the agent with the question prompt
        # Adjust the API call if your agent uses a different method name
        agent_output = agent.run(question)

        # Determine correctness via exact match (you can change this evaluation)
        is_correct = (agent_output.strip().lower() == ground_truth.strip().lower())
        if is_correct:
            correct += 1

        results.append({
            "task_id": example.get("task_id"),
            "question": question,
            "ground_truth": ground_truth,
            "agent_output": agent_output,
            "is_correct": is_correct
        })

    accuracy = correct / total if total > 0 else 0
    eval_summary = {
        "total_examples": total,
        "correct": correct,
        "accuracy": accuracy,
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    }

    # Save the evaluation results to the specified JSON file.
    evaluation_data = {
        "summary": eval_summary,
        "results": results
    }
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(evaluation_data, f, ensure_ascii=False, indent=2)

    print(f"\nEvaluation complete. Accuracy: {accuracy * 100:.2f}%")
    print(f"Detailed results saved to: {output_file}")

In [23]:
test_split = dataset["test"]

In [24]:
agent = setup_agent()

Created Persona: Eddie
[Eddie] Tool: added - combination_calculator 
[Eddie] Tool: added - add 
[Eddie] Tool: added - subtract 
[Eddie] Tool: added - multiply 
[Eddie] Tool: added - divide 


In [13]:
time.sleep(2)

In [25]:
output_file = "evaluation_results.json"
evaluate_agent(agent, test_split, output_file)

Evaluating:   0%|                                       | 0/301 [00:00<?, ?it/s]

[Eddie] ++++++++++Cycle Started++++++++++
[Eddie][Trigger: Input]: Here's a fun riddle that I'd like you to try.

An adventurer exploring an ancient tomb came across a horde of gold coins, all neatly stacked in columns. As he reached to scoop them into his backpack, a mysterious voice filled the room. "You have fallen for my trap adventurer," the voice began, and suddenly the doorway to the chamber was sealed by a heavy rolling disk of stone. The adventurer tried to move the stone disk but was unable to budge the heavy stone. Trapped, he was startled when the voice again spoke. 

"If you solve my riddle, I will reward you with a portion of my riches, but if you are not clever, you will never leave this treasure chamber. Before you are 200 gold coins. I pose a challenge to you, adventurer. Within these stacks of coins, all but 30 are face-up. You must divide the coins into two piles, one is yours, and one is mine. You may place as many coins as you like in either pile. You may flip any 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Evaluating:   0%|                               | 1/301 [00:04<24:02,  4.81s/it]

[Eddie][Response]: The adventurer created a pile of 30 coins and flipped each one. If the larger pile had 14 face-down coins, then the smaller pile of 30 coins also must have had 14 face-down coins. This means that the adventurer successfully created two piles with the same number of face-down coins. Since each face-down coin earned him two gold coins, he won 28 gold coins.

FINAL ANSWER: 28
[Eddie] ----------Cycle Ended------------
[Eddie] ++++++++++Cycle Started++++++++++
[Eddie][Trigger: Input]: What animal is shown on page 54 of Anatomy and Physiology of Animals by Ruth Lawson in Dirk Hünniger's 2015 pdf conversion? Just give the common name, without any article.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Evaluating:   1%|▏                              | 2/301 [00:07<17:17,  3.47s/it]

...

[Eddie][Response]: I'm sorry, I don't have access to the specific edit history of the English Wikipedia page for Fiji Mermaids to determine how many times it was edited on April Fool's Day between 2000 and 2022. You may need to visit the Wikipedia page's history directly for this information. 

FINAL ANSWER: Cannot compute without access to the Wikipedia page history.
[Eddie] ----------Cycle Ended------------
[Eddie] ++++++++++Cycle Started++++++++++
[Eddie][Trigger: Input]: Train an AI assistant able to solve GAIA (see https://huggingface.co/gaia-benchmark).


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Evaluating: 100%|█████████████████████████████| 301/301 [15:02<00:00,  2.93s/it]

[Eddie][Response]: I understand that you want to train an AI assistant to solve GAIA, a benchmark available on Hugging Face. To achieve that, you'll need access to the necessary datasets and tools required for the specific tasks in GAIA. The benchmark involves evaluating AI models across different natural language processing tasks, often including tasks like reading comprehension, language understanding, and generation.

Given the lack of a specific question in your prompt, it's difficult to give detailed instructions on this process without further clarification. If you have specific tasks or questions regarding GAIA, please feel free to ask.

FINAL ANSWER: Train AI for GAIA benchmark by accessing datasets and utilizing natural language processing tools.


Evaluating: 100%|█████████████████████████████| 301/301 [15:02<00:00,  3.00s/it]

[Eddie] ----------Cycle Ended------------

Evaluation complete. Accuracy: 0.00%
Detailed results saved to: evaluation_results.json


In [17]:
import re
import string
import logging
from typing import Any, List, Optional

# Set up a basic logger for demonstration.
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Evaluator:
    def question_scorer(self, model_answer: str, ground_truth: str) -> bool:
        r"""Scorer for the GAIA benchmark.
        https://huggingface.co/spaces/gaia-benchmark/leaderboard/blob/main/scorer.py

        Args:
            model_answer (str): The model answer.
            ground_truth (str): The ground truth answer.

        Returns:
            bool: True if the normalized answer matches the ground truth.
        """
        def is_float(element: Any) -> bool:
            try:
                float(element)
                return True
            except ValueError:
                return False

        if is_float(ground_truth):
            logger.info(f"Evaluating '{model_answer}' as a number.")
            normalized_answer = self.normalize_number_str(model_answer)
            return normalized_answer == float(ground_truth)

        elif any(char in ground_truth for char in [",", ";"]):
            logger.info(f"Evaluating '{model_answer}' as a comma separated list.")
            gt_elems = self.split_string(ground_truth)
            ma_elems = self.split_string(model_answer)

            if len(gt_elems) != len(ma_elems):
                logger.warning(
                    "Answer lists have different lengths, returning False."
                )
                return False

            comparisons = []
            for ma_elem, gt_elem in zip(ma_elems, gt_elems):
                if is_float(gt_elem):
                    normalized_ma_elem = self.normalize_number_str(ma_elem)
                    comparisons.append(normalized_ma_elem == float(gt_elem))
                else:
                    ma_elem = self.normalize_str(ma_elem, remove_punct=False)
                    gt_elem = self.normalize_str(gt_elem, remove_punct=False)
                    comparisons.append(ma_elem == gt_elem)
            return all(comparisons)
        else:
            logger.info(f"Evaluating '{model_answer}' as a string.")
            ma_elem = self.normalize_str(model_answer)
            gt_elem = self.normalize_str(ground_truth)
            return ma_elem == gt_elem

    def normalize_number_str(self, number_str: str) -> float:
        for char in ["$", "%", ","]:
            number_str = number_str.replace(char, "")
        try:
            return float(number_str)
        except ValueError:
            logger.error(f"String '{number_str}' cannot be normalized to number.")
            return float("inf")

    def split_string(self, s: str, char_list: Optional[List[str]] = None) -> List[str]:
        r"""Split a string based on a list of characters.

        Args:
            s (str): The string to split.
            char_list (Optional[List[str]], optional):
                The list of characters to split on.
                (default: None)

        Returns:
            List[str]: The list of split elements.
        """
        if char_list is None:
            char_list = [",", ";"]
        pattern = f"[{''.join(char_list)}]"
        # Filter out empty strings that might occur after splitting
        return [elem.strip() for elem in re.split(pattern, s) if elem.strip()]

    def normalize_str(self, input_str: str, remove_punct: bool = True) -> str:
        r"""Normalize a string by removing spaces, setting to lower-case,
        and optionally removing punctuation.

        Args:
            input_str (str): The input string to normalize.
            remove_punct (bool, optional): Whether to remove punctuation. Defaults to True.

        Returns:
            str: The normalized string.
        """
        no_spaces = re.sub(r"\s", "", input_str)
        if remove_punct:
            translator = str.maketrans("", "", string.punctuation)
            return no_spaces.lower().translate(translator)
        else:
            return no_spaces.lower()

In [18]:
# Create an instance of the evaluator
evaluator = Evaluator()

# Example 1: Numeric answer evaluation.
ground_truth_num = "42"
model_answer_num = "42"  # Should score True
score_num = evaluator.question_scorer(model_answer_num, ground_truth_num)
print("Numeric Evaluation:", score_num)

# Example 2: Comma-separated list evaluation.
ground_truth_list = "3, 5, 7"
model_answer_list = "3,5,7"  # Should score True because normalization ignores spaces
score_list = evaluator.question_scorer(model_answer_list, ground_truth_list)
print("List Evaluation:", score_list)

# Example 3: String evaluation.
ground_truth_str = "This is a test answer."
model_answer_str = "this is a test answer"  # Should score True after normalization
score_str = evaluator.question_scorer(model_answer_str, ground_truth_str)
print("String Evaluation:", score_str)

INFO:__main__:Evaluating '42' as a number.
INFO:__main__:Evaluating '3,5,7' as a comma separated list.
INFO:__main__:Evaluating 'this is a test answer' as a string.


Numeric Evaluation: True
List Evaluation: True
String Evaluation: True


In [19]:
# You can also test with your custom examples. For instance:
ground_truth_custom = "1000"
model_answer_custom = "$1,000"  # Even with symbols and commas, it should normalize correctly.
score_custom = evaluator.question_scorer(model_answer_custom, ground_truth_custom)
print("Custom Numeric Evaluation:", score_custom)

INFO:__main__:Evaluating '$1,000' as a number.


Custom Numeric Evaluation: True
